<a href="https://colab.research.google.com/github/ugoGS/Py/blob/main/generate_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def generate_csv(table_name):

  df = get_df_last_partition(refined_path, table_name, 'process_datetime')

  target_path = f"{delivery_path}/lsmw_files/{process_datetime}"

  fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
  path = spark._jvm.org.apache.hadoop.fs.Path(target_path)

  if not fs.exists(path):
    fs.mkdirs(path)

  df.coalesce(1).write.mode("append").option("header", "true").csv(f"{target_path}/{table_name}_{process_datetime}")

In [ ]:
# def generate_csv_lsmw(df, table_name):

#   target_path = f"{delivery_path}/lsmw_files/{process_datetime}"

#   fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
#   path = spark._jvm.org.apache.hadoop.fs.Path(target_path)

#   if not fs.exists(path):
#     fs.mkdirs(path)

#  #HGS 16/01/2025
#   #table_name = table_name.upper()

#   df.coalesce(1).write.mode("append").option("header", "true").csv(f"{target_path}/{table_name}_{process_datetime}")

In [ ]:

def generate_csv_lsmw(df, table_name):

  table_name_upper = table_name.upper()

  target_path = f"{delivery_path}/lsmw_files/{process_datetime}/{table_name}_{process_datetime}"
  csv_name = f"{table_name_upper}_{process_datetime}.csv"
  full_csv_path = f"{target_path}/{csv_name}"

  # Crear directorio si no existe
  fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
  path = spark._jvm.org.apache.hadoop.fs.Path(target_path)

  if not fs.exists(path):
    fs.mkdirs(path)

  # Verificar si ya existe un archivo CSV para el mismo nombre
  if os.path.exists(full_csv_path):
    # Leer archivo CSV existente usando pandas
    existing_df = pd.read_csv(full_csv_path)
    new_df = df.toPandas()  # Convertir DataFrame de Spark a pandas
    combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    #HGS 23/01/2025
    combined_df = combined_df.drop_duplicates()  # Drop dulicates
    combined_df.to_csv(full_csv_path, index=False)

  else:
    # Crear nuevo archivo CSV si no existe
    df.coalesce(1).write.mode("overwrite").option("header", "true").csv(f"{target_path}") #/{table_name}_{process_datetime}")


In [ ]:
def generate_csv_df(table_name, path):

  df = get_df_last_partition(path, table_name, 'process_datetime')

  target_path = f"{delivery_path}/other_files/{process_datetime}"

  fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
  path = spark._jvm.org.apache.hadoop.fs.Path(target_path)

  if not fs.exists(path):
    fs.mkdirs(path)

  df.coalesce(1).write.mode("append").option("header", "true").csv(f"{target_path}/{table_name}_{process_datetime}")

In [ ]:
def rename_csv_file(table_name):

  try:
    table_name_upper = table_name.upper()

    csv_name = f"{table_name_upper}_{process_datetime}.csv"
    target_path = f"{delivery_path}/lsmw_files/{process_datetime}/{table_name}_{process_datetime}"

    files = os.listdir(target_path)
    # Iterator (find firts available condition)
    csv_file = next((file for file in files if file.startswith("part-") and file.endswith(".csv")), None)

    if csv_file:

      old_file_path = os.path.join(target_path, csv_file)
      new_file_path = os.path.join(target_path, csv_name)

      # Rename the file
      shutil.move(old_file_path, new_file_path)

      return True

    else:
      return False

  except Exception as e:
    print(f"[rename_csv_file] Unexpected error: {str(e)}")
    return False
